In [1]:


#pandas libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import random

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import log_loss
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import Normalizer

from scipy.stats import uniform

%run "../notebooks/cargar_df.ipynb"

### Cargo los data frames con los features

In [2]:
trainX = pd.read_csv('../files_csv/train_173_features_bin_encoding')
trainX.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD,Stage
0,0,1,1,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0.52,0.58817,5.609362e+06,0
1,1,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0.53,0.59948,5.130851e+04,1
2,2,0,0,1,1,0,1,0,1,0,...,1,0,0,0,1,0,0.48,0.48000,8.386560e+04,1
3,3,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53,0.53000,7.421882e+06,0
4,4,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53,0.53000,1.335719e+07,0


In [3]:
stages = trainX['Stage']
stages.head()

0    0
1    1
2    1
3    0
4    0
Name: Stage, dtype: int64

In [4]:
ids = trainX.Opportunity_ID
ids.head()

0    0
1    1
2    2
3    3
4    4
Name: Opportunity_ID, dtype: int64

In [5]:
trainX = trainX.drop(['Opportunity_ID', 'Stage'], axis=1)

In [6]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 12.8 MB


In [7]:
testX = pd.read_csv('../files_csv/test_173_features_bin_encoding')
testX.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_1,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD
0,10689,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.306667,0.346870,137147.816349
1,10690,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.332000,0.375524,169716.349384
2,10691,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.425000,0.425000,21037.500000
3,10692,1,1,1,0,1,1,0,0,1,...,0,1,0,0,0,1,0,0.425000,0.425000,361517.750000
4,10693,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.390000,0.390000,5752.500000


In [8]:
ids_test = testX.Opportunity_ID
ids_test.head()

0    10689
1    10690
2    10691
3    10692
4    10693
Name: Opportunity_ID, dtype: int64

In [9]:
testX = testX.drop('Opportunity_ID', axis=1)

In [10]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 2.1 MB


# Perceptron

Version Inicial

In [11]:
# Del archivo 'prediccion_log_regression'
# Las columnas mas influyentes para predecir con Logistic Regression

test_size = 0.33
random_state = 0

In [12]:
datos_train , datos_test, cat_train, cat_test = train_test_split(trainX,
                                                               stages,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [13]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)

In [14]:
n_iter=400
eta0 = 0.1
shuffle=False

In [15]:
#ppn con todos los features
ppn = Perceptron(max_iter = n_iter,
                eta0 = eta0,
                random_state = random_state,
                 shuffle = shuffle)

ppn.fit(datos_train_std,cat_train)

Perceptron(eta0=0.1, max_iter=400, shuffle=False)

In [16]:
cat_pred = ppn.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 70.88%


In [17]:
final_test_data = testX[trainX.columns]

In [18]:
preds = ppn.predict(final_test_data)
preds

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [19]:
prediccion = pd.DataFrame({'Opportunity_ID':ids_test, 'Target':pd.DataFrame(preds)[0]})
prediccion.head()

,Opportunity_ID,Target
0,10689,0
1,10690,0
2,10691,0
3,10692,0
4,10693,0


In [20]:
prediccion.to_csv('prediccion_perceptron', index=False)

# Busqueda de hiperparametros optimos

In [42]:

cv=StratifiedKFold(random_state=None)
gs = GridSearchCV(ppn, 
                  {'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3], 'max_iter': [5, 10, 15, 20, 50]},
                  cv=cv, 
                  scoring='f1')


gs = gs.fit(trainX.iloc[:, 1:], trainX.iloc[:, 0])

E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:629: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anacond

E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
E:\PythonFiles\Anaconda3\lib\site-packages\s

In [45]:
print(gs.best_score_)
print(gs.best_params_)

0.8213543672891328
{'alpha': 0.0001, 'max_iter': 5}


Aplicacion de parametros optimos

In [23]:
alpha =  0.0001
max_iter = 5

In [24]:
#ppn con todos los features
ppn = Perceptron(max_iter = max_iter,
                eta0 = eta0,
                random_state = random_state,
                shuffle = shuffle,
                alpha = alpha)

ppn.fit(datos_train_std,cat_train)
cat_pred = ppn.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 73.24%


C:\Users\Alan\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [50]:
final_test_data = testX[trainX.columns]
preds = ppn.predict(final_test_data)
prediccion = pd.DataFrame({'Opportunity_ID':ids_test, 'Target':pd.DataFrame(preds)[0]})
prediccion.to_csv('prediccion_perceptron_op', index=False)

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

# Removiendo Leakage por Model Stacking

In [61]:
RANDOM_STATE = 12345

ppt = Perceptron(random_state=RANDOM_STATE)
knn = KNeighborsClassifier(n_neighbors=3)
rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=RANDOM_STATE)

In [62]:
df_X = trainX
df_y = stages

ppt.fit(df_X, df_y)
knn.fit(df_X, df_y)
rf.fit(df_X, df_y)

RandomForestClassifier(max_depth=3, random_state=12345)

In [63]:
X_ppt = ppt.predict(df_X)
X_knn = knn.predict(df_X)
X_rf = rf.predict(df_X)

In [64]:
X_level2 = np.vstack([X_ppt, X_knn, X_rf]).T

In [65]:
X_level2.shape

(9791, 3)

# Prediccion de 2do nivel

In [68]:
lr = LogisticRegression(random_state=RANDOM_STATE)

In [69]:
cv_level2 = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

In [70]:
y_pred = cross_val_predict(lr, X_level2, df_y, cv=cv_level2)

In [71]:
auc_before = roc_auc_score(y_pred, df_y)

In [88]:
print('AUC original:', auc_before)

AUC original: 0.8364113800211359


# Removiendo Leakeage

In [74]:

cv_level1 = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE * 2)
cv_level2 = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

In [75]:
ppt = Perceptron(random_state=RANDOM_STATE)
knn = KNeighborsClassifier(n_neighbors=3)
rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=RANDOM_STATE)

In [76]:
X_ppt_oof = cross_val_predict(ppt, df_X, df_y, cv=cv_level1)
X_knn_oof = cross_val_predict(knn, df_X, df_y, cv=cv_level1)
X_rf_oof = cross_val_predict(rf, df_X, df_y, cv=cv_level1)

In [77]:

X_level2 = np.vstack([X_ppt_oof, X_knn_oof, X_rf_oof]).T

In [78]:
X_level2.shape

(9791, 3)

# Prediccion de Segundo nivel

In [81]:
lr = LogisticRegression(random_state=RANDOM_STATE)


In [82]:
y_pred = cross_val_predict(lr, X_level2, df_y, cv=cv_level2)


In [83]:
auc_after = roc_auc_score(y_pred, df_y)


In [87]:
print('AUC final:', auc_after)


AUC final: 0.7802100939728337


# Evaluacion final de Leakage

In [86]:
print('AUC difference:', auc_after - auc_before)


AUC difference: -0.05620128604830221
